In [1]:
import os
import pandas as pd
import numpy as np

# Loading Data_Info

In [4]:
files = os.listdir('../../patches/')

In [5]:
files = pd.DataFrame(data=files, columns=['filename'])

In [6]:
x = files["filename"].str.split("_", n = 3, expand = True)
y = x[x.columns[3]].str.split(".", n = 1, expand = True)
files["SVS_NO"] = x[x.columns[0]]
files["Class_Name"] = y[y.columns[0]]
files["Class_Name"] = files["Class_Name"].astype('int')

In [7]:
files['filename'] = '../../patches/' + files['filename']

In [8]:
files.head()

,filename,SVS_NO,Class_Name
0,../../patches/A08_34_65_0.png,A08,0
1,../../patches/A01_145_118_1.png,A01,1
2,../../patches/A09_246_94_0.png,A09,0
3,../../patches/A08_78_10_0.png,A08,0
4,../../patches/A10_80_79_3.png,A10,3


In [9]:
print ("Total_Patches : ", files.shape[0])

Total_Patches :  359487


In [10]:
all_avalable_svs = np.unique(files.SVS_NO.astype('str'), return_counts=True)

In [11]:
all_avalable_svs

(array(['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09',
        'A10'], dtype=object),
 array([40016, 38340, 34048, 44856, 33366, 34892, 28171, 36480, 33750,
        35568]))

In [12]:
Xfiles = os.listdir('../../Dataset/X1_P/')
Xfiles = pd.DataFrame(data=Xfiles, columns=['filename'])
x = Xfiles["filename"].str.split("_", n = 3, expand = True)
y = x[x.columns[3]].str.split(".", n = 1, expand = True)
Xfiles["SVS_NO"] = x[x.columns[0]]
Xfiles["Class_Name"] = y[y.columns[0]]
Xfiles["Class_Name"] = Xfiles["Class_Name"].astype('int')
Xfiles['filename'] = '../../Dataset/X1_P/' + Xfiles['filename']
print ("Total_Patches : ", Xfiles.shape[0])
Xfiles.head()

Total_Patches :  15544


,filename,SVS_NO,Class_Name
0,../../Dataset/X1_P/TCGA-A2-A0CP-01Z-00-DX1.ECF...,TCGA-A2-A0CP-01Z-00-DX1.ECFD263C-BB17-4ADA-8F2...,0
1,../../Dataset/X1_P/TCGA-A2-A0CP-01Z-00-DX1.ECF...,TCGA-A2-A0CP-01Z-00-DX1.ECFD263C-BB17-4ADA-8F2...,0
2,../../Dataset/X1_P/TCGA-A2-A0CP-01Z-00-DX1.ECF...,TCGA-A2-A0CP-01Z-00-DX1.ECFD263C-BB17-4ADA-8F2...,0
3,../../Dataset/X1_P/TCGA-A2-A0CP-01Z-00-DX1.ECF...,TCGA-A2-A0CP-01Z-00-DX1.ECFD263C-BB17-4ADA-8F2...,0
4,../../Dataset/X1_P/TCGA-A2-A0CP-01Z-00-DX1.ECF...,TCGA-A2-A0CP-01Z-00-DX1.ECFD263C-BB17-4ADA-8F2...,0


# Setting up Dryad Label for Invasive and Normal

In [13]:
files['Class_Name'][files['Class_Name'] > 0] = 1

/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [14]:
Xfiles['Class_Name'][Xfiles['Class_Name'] == -1] = 1

/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [15]:
Xfiles.Class_Name.unique()

array([0, 1])

In [16]:
np.unique(files.SVS_NO.astype('str'))

array(['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09',
       'A10'], dtype=object)

In [17]:
training_all_avalable_svs = np.unique(files.SVS_NO.astype('str'), return_counts=True)
training_all_avalable_svs

(array(['A01', 'A02', 'A03', 'A04', 'A05', 'A06', 'A07', 'A08', 'A09',
        'A10'], dtype=object),
 array([40016, 38340, 34048, 44856, 33366, 34892, 28171, 36480, 33750,
        35568]))

In [18]:
print ("Training Total_Patches : ", files.shape[0])

Training Total_Patches :  359487


# S-Score Calculation (Metric used in ICIAR 2018)

In [19]:
def vectorized_s_score(x, y):
    a = 0.0000000001
    nom = np.sum(np.abs(x-y))
    xbin = x.copy()
    ybin = y.copy()
    xbin[xbin!=0] = 1.0
    ybin[ybin!=0] = 1.0
    temp = 1.0 - ((1.0 - xbin) * (1.0 - ybin))
    v = np.abs(y - 3)
    max_term = np.max(np.column_stack([y, v]), axis=1)
    temp1 = max_term * temp
    denom = np.sum(temp1)
    final = nom / (denom + a)
    return (1 - final)

# Loading Libraries for Training

In [20]:
import os
import json
import cv2
import numpy as np
import pandas as pd
import keras
import pandas as pd
import numpy as np
import os
from glob import glob
import itertools
import fnmatch
import random
import matplotlib.pylab as plt
import cv2
from scipy.misc import imresize, imread
import sklearn
from sklearn import model_selection
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, learning_curve, GridSearchCV
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import keras
from keras import layers
from tqdm import tqdm
from keras import backend as K
from keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, model_from_json,load_model,Model
from keras.optimizers import SGD, RMSprop, Adam, Adagrad, Adadelta
from keras.utils import layer_utils
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, UpSampling2D , Conv2D, MaxPool2D, MaxPooling2D,Input,Concatenate,GlobalAveragePooling2D,GlobalMaxPooling2D,ZeroPadding2D,AveragePooling2D,Reshape,merge,Convolution2D
%matplotlib inline
from keras import layers
from keras.applications import MobileNetV2
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.optimizers import Adam
import tensorflow as tf
from tqdm import tqdm
import efficientnet.keras as efn 
from keras.layers import Dense
from keras.optimizers import Adam, Nadam
from keras_radam import RAdam
from keras import backend as K
from sklearn.utils import class_weight

Using TensorFlow backend.
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/qali/anaconda3/envs/py37_gpu_env/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/qa

# Training

In [21]:
BATCH_SIZE = 32

def create_datagen():
    return ImageDataGenerator(
        zoom_range=0.1,  # set range for random zoom
        fill_mode='constant',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images,
        rescale=1/255.,
#        validation_split=0.15
    )

In [22]:
def create_flow(datagen, subset=None):
    return datagen.flow_from_dataframe(
        files, 
        directory='.',
        x_col='filename', 
        y_col='Class_Name', 
        class_mode='binary',
        target_size=(256, 256),
        batch_size=BATCH_SIZE,
#        subset=subset
    )


In [23]:
#To set class_mode of generator we need to change label from dtype integer to dtype string
files['Class_Name'] = files['Class_Name'].astype('str')

In [24]:
data_generator = create_datagen()
train_gen = create_flow(data_generator, 'training')

Found 359487 validated image filenames belonging to 2 classes.


In [25]:
def testcreate_flow(datagen, subset=None):
    return datagen.flow_from_dataframe(
        Xfiles, 
        directory='.',
        x_col='filename', 
        y_col='Class_Name', 
        class_mode='binary',
        target_size=(256, 256),
        batch_size=BATCH_SIZE,
#        subset=subset
    )


In [26]:
#To set class_mode of generator we need to change label from dtype integer to dtype string
Xfiles['Class_Name'] = Xfiles['Class_Name'].astype('str')
val_gen = testcreate_flow(data_generator, 'validation')

Found 15544 validated image filenames belonging to 2 classes.


In [27]:
efficient_net = efn.EfficientNetB0(weights='imagenet',input_shape=(256,256,3),include_top=False,pooling='max')

In [28]:
efficient_net.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 256, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 128, 128, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 128, 128, 32) 128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 128, 128, 32) 0           stem_bn[0][0]                    
__________________________________________________________________________________________________
block1a_dw

In [29]:
model = Sequential()
model.add(efficient_net)
model.add(Dense(units = 128, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dense(units = 1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 1280)              4049564   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               163968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 4,221,853
Trainable params: 4,179,837
Non-trainable params: 42,016
_________________________________________________________________


In [30]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [31]:
model.compile(optimizer=RAdam(), loss='binary_crossentropy', metrics=['accuracy', f1_m, precision_m, recall_m])

In [32]:
class_weights = class_weight.compute_class_weight('balanced',
                                                 np.unique(files.Class_Name),
                                                 files.Class_Name)

In [33]:
class_weights, np.unique(files.Class_Name, return_counts=True)

(array([0.66306196, 2.03315951]),
 (array(['0', '1'], dtype=object), array([271081,  88406])))

In [ ]:
total_steps = files.shape[0] / BATCH_SIZE
test_total_steps = Xfiles.shape[0] / BATCH_SIZE

checkpoint = ModelCheckpoint('Complete_model_effnet_b0_5.h5', monitor='val_f1_m', verbose=1, save_best_only=True, save_weights_only=False,mode='max')
history = model.fit_generator(train_gen,steps_per_epoch=total_steps, validation_data=val_gen,
    validation_steps=test_total_steps,epochs=25,callbacks=[checkpoint], class_weight=class_weights)

Epoch 1/25
11234/11233 [==============================] - 4498s 400ms/step - loss: 0.1890 - acc: 0.9342 - f1_m: 0.8522 - precision_m: 0.8956 - recall_m: 0.8323 - val_loss: 0.2051 - val_acc: 0.9234 - val_f1_m: 0.6173 - val_precision_m: 0.8204 - val_recall_m: 0.5314

Epoch 00001: val_f1_m improved from -inf to 0.61726, saving model to Complete_model_effnet_b0_5.h5
Epoch 2/25
11234/11233 [==============================] - 4520s 402ms/step - loss: 0.1549 - acc: 0.9456 - f1_m: 0.8780 - precision_m: 0.9178 - recall_m: 0.8574 - val_loss: 0.1963 - val_acc: 0.9145 - val_f1_m: 0.5271 - val_precision_m: 0.8260 - val_recall_m: 0.4126

Epoch 00002: val_f1_m did not improve from 0.61726
Epoch 3/25
11234/11233 [==============================] - 4489s 400ms/step - loss: 0.1458 - acc: 0.9487 - f1_m: 0.8848 - precision_m: 0.9260 - recall_m: 0.8618 - val_loss: 0.1528 - val_acc: 0.9351 - val_f1_m: 0.7039 - val_precision_m: 0.8350 - val_recall_m: 0.6492

Epoch 00003: val_f1_m improved from 0.61726 to 0.703

In [ ]:
np.unique(Xfiles.Class_Name.values, return_counts=True)

In [27]:
model = load_model('Complete_model_effnet_b0_5.h5', compile=False)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnet-b0 (Model)      (None, 1280)              4049564   
_________________________________________________________________
dense_1 (Dense)              (None, 128)               163968    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 4,221,853
Trainable params: 4,179,837
Non-trainable params: 42,016
_________________________________________________________________


# TEST

In [28]:
BATCH_SIZE = 1
def create_test_gen():
    return ImageDataGenerator(rescale=1/255.).flow_from_dataframe(
        Xfiles,
        directory='.',
        x_col='filename',
        class_mode=None,
        target_size=(256, 256),
        batch_size=BATCH_SIZE,
        shuffle=False
    )

test_gen = create_test_gen()

Found 15544 validated image filenames.


In [29]:
predictions = model.predict_generator(test_gen, steps=len(test_gen), verbose=1)

15544/15544 [==============================] - 130s 8ms/step


In [50]:
#pred = np.argmax(predictions, axis=1)
pred = predictions.copy()
pred[pred > 0.5] = 1
pred[pred <= 0.5] = 0

In [51]:
accuracy_score(Xfiles.Class_Name.astype('int'), pred)

0.9379181677817807

In [60]:
f1_score(Xfiles.Class_Name.astype('int'), pred.astype('int'), average='micro')

0.9379181677817807

In [62]:
f1_score(Xfiles.Class_Name.astype('int'), pred.astype('int'), average='macro')

0.8597955419106967

In [61]:
f1_score(Xfiles.Class_Name.astype('int'), pred.astype('int'), average='weighted')

0.9353933048410192

In [63]:
confusion_matrix(Xfiles.Class_Name.astype('int'), pred.astype('int'))

array([[13091,   295],
       [  670,  1488]])

# Saving predictions

In [64]:
Xfiles['predictions_epochs_3'] = predictions.copy()
Xfiles.to_csv('A0CP_Predictions.csv', index=False)

In [21]:
df = pd.read_csv('A0CP_Predictions.csv')
x = df.predictions_epochs_3.values
y = df.Class_Name.values
score = vectorized_s_score(x, y)
print ("S-Score : ", score)

S-Score :  0.9680307023287101
